# STEP 1

## Player class

In [1]:
import sys
import random

#The class is used to better perform the AVL tree operations
class Player(object):
    def __init__(self, player):
        self.player = player
        self.name = player["playername"]
        self.score = float(player["score"])
        self.games = player["games"]
        self.left = None
        self.right = None
        self.height = 1

    # returns the mean score of all the players games at anytime
    def return_mean_score(self):
        score = 0
        for game in self.games:
            score+=game
        score = score/len(self.games)
        return score

    #Display function given by our professor
    def display(self):
      lines, *_ = self._display_aux()
      for line in lines:
          print(line)

    #Display function given by our professor
    def _display_aux(self):
        """Returns list of strings, width, height, and horizontal coordinate of the root."""
        # No child.
        if self.right is None and self.left is None:
            line = '%s' % self.score
            width = len(line)
            height = 1
            middle = width // 2
            return [line], width, height, middle

        # Only left child.
        if self.right is None:
            lines, n, p, x = self.left._display_aux()
            s = '%s' % self.score
            u = len(s)
            first_line = (x + 1) * ' ' + (n - x - 1) * '_' + s
            second_line = x * ' ' + '/' + (n - x - 1 + u) * ' '
            shifted_lines = [line + u * ' ' for line in lines]
            return [first_line, second_line] + shifted_lines, n + u, p + 2, n + u // 2

        # Only right child.
        if self.left is None:
            lines, n, p, x = self.right._display_aux()
            s = '%s' % self.score
            u = len(s)
            first_line = s + x * '_' + (n - x) * ' '
            second_line = (u + x) * ' ' + '\\' + (n - x - 1) * ' '
            shifted_lines = [u * ' ' + line for line in lines]
            return [first_line, second_line] + shifted_lines, n + u, p + 2, u // 2
            # Two children.
        left, n, p, x = self.left._display_aux()
        right, m, q, y = self.right._display_aux()
        s = '%s' % self.score
        u = len(s)
        first_line = (x + 1) * ' ' + (n - x - 1) * '_' + s + y * '_' + (m - y) * ' '
        second_line = x * ' ' + '/' + (n - x - 1 + u + y) * ' ' + '\\' + (m - y - 1) * ' '
        if p < q:
            left += [n * ' '] * (q - p)
        elif q < p:
            right += [m * ' '] * (p - q)
        zipped_lines = zip(left, right)
        lines = [first_line, second_line] + [a + u * ' ' + b for a, b in zipped_lines]
        return lines, n + m + u, max(p, q) + 2, n + u // 2

# AVL Tree + specific to problem functions

In [3]:
class AVLTree(object):
    #Basic AVL Tree operations
    def insererNoeud(self, root, player):
        
        # trouve le bon endroit pour l'insérer
        if not root:
            return player
        elif player.score < root.score:
            root.left = self.insererNoeud(root.left, player)
        else:
            root.right = self.insererNoeud(root.right, player)

        root.height = 1 + max(self.getHauteur(root.left), self.getHauteur(root.right))

        # met à jour la balance et fais la balance
        facteurBalance = self.getBalance(root)
        if facteurBalance > 1:
            if player.score <= root.left.score:
                return self.rotationDroite(root)
            else:
                root.left = self.rotationGauche(root.left)
                return self.rotationDroite(root)

        if facteurBalance < -1:
            if player.score >= root.right.score:
                return self.rotationGauche(root)
            else:
                root.right = self.rotationDroite(root.right)
                return self.rotationGauche(root)

        return root
    
    def getHauteur(self, root):
        if not root:
            return 0
        return root.height
    
    def getBalance(self, root):
        if not root:
            return 0
        return self.getHauteur(root.left) - self.getHauteur(root.right)
    
    
    def getMinValueNode(self, root):
        if root is None or root.left is None:
            return root
        return self.getMinValueNode(root.left)


    def rotationGauche(self, z):
        y = z.right
        T2 = y.left
        y.left = z
        z.right = T2
        z.height = 1 + max(self.getHauteur(z.left), self.getHauteur(z.right))
        y.height = 1 + max(self.getHauteur(y.left), self.getHauteur(y.right))
        return y

    def rotationDroite(self, z):
        #print("z",z.player.name)
        y = z.left
        #print("y",y.player.name)
        T3 = y.right
        y.right = z
        z.left = T3
        z.height = 1 + max(self.getHauteur(z.left), self.getHauteur(z.right))
        y.height = 1 + max(self.getHauteur(y.left), self.getHauteur(y.right))
        return y

    def estBalance(self, root):
        if root is None:
            return True
        lh = self.getHauteur(root.left)
        rh = self.getHauteur(root.right)
        if (abs(lh - rh) and self.getBalance(root.left) <= 1 and self.getBalance(root.left) <= 1 ):
            return True
        return False

    



#SPECIFIC FUNCTIONS FOR THE PROBLEM

# unused function since the tree isn't sorted after using it, and the time complexity for that would be worst than the method we use.     
# It responds to the question 3. only
# if we would use it, then we should respond to the question 4. and, by using getAllPlayersFromAVL() and browsing the tree with randomized score, to insert the values in the new AVL Tree.
    '''
    def updateScoresRandomly(self, root):    
      if not root:    
          return   
      value = random.randrange(0, 13, 1)
      #print(value)
      root.games.append(value) 
      root.score = meanScore(root.games) 
      self.updateScoresRandomly(root.left) 
      self.updateScoresRandomly(root.right)
      return root
    '''


# updates the score of an avl tree, and add the node in another avl tree in order to have it stored and sorted without using a list
# used for question 3/4
    def updateScoresRandomlyAndInsertInNewTree(self, root, newtree, root2):    
      if not root: 
          return   

      value = random.randrange(0, 13, 1) # We create the new score
      root.games.append(value) #update its games list
      root.score = meanScore(root.games)  # And its score
      root2 = newtree.insererNoeud(root2, Player({'playername' : root.name, 'score' : root.score, 'games' : root.games})) # Then we add it to the new AVL Tree so that its sorted   

      self.updateScoresRandomlyAndInsertInNewTree(root.left, newtree, root2) # And then we continue browsing every node of the old tree
      self.updateScoresRandomlyAndInsertInNewTree(root.right, newtree, root2)

      return root2, newtree







#HERE ARE ALL THE FUNCTIONS THAT WE USE


    # Is a O(n) function which browse every nodes of the AVL Tree with pre-order traversal
    def getAllPlayersFromAVL(self, root, players):
      if not root: 
          return   
      players.append(root.player) 
      self.getAllPlayersFromAVL(root.left, players) 
      self.getAllPlayersFromAVL(root.right, players)       
      return players

    def createSortedPlayers(self, root, games): #is a in-order traversal
      if not root: 
          return        
      self.createSortedPlayers(root.left, games)
      games.append(root.player) 
      self.createSortedPlayers(root.right, games)  
      return games



    # creates games randomly based on the players in the AVL Tree
    def createRandomGames(self, root):
      players = self.getAllPlayersFromAVL(root, [])
      listeJ = init_list_of_objects(int(len(players)/10)) #Create a list of list of the number of games that will be played (ex : for 90 players => 9 games, so a list of 9 lists)
      indices = []
      for i in range(len(players)): #list of possible positions  
        indices.append(i)
      for u in range(int(len(players)/10)): #We move in each list inside listeJ
        for i in range(10): #For each of the 10 players we will insert in each liste 
          ind = random.randrange(0, len(indices), 1) # We select a random player
          player = players[indices.pop(ind)] # we select the player we chose and we delete its indices so that we won't take this player again
          listeJ[u].append(player) # We add it into a game
      return listeJ 

    # creates games based on the rankings of the players in the AVL Tree using the natural sorting capabilities of an AVL Tree (In order traversal)
    def createRankedgames(self, root):
      players = self.createSortedPlayers(root, []) #In order traversal of the Tree
      listGames = init_list_of_objects(int(len(players)/10))
      for u in range(int(len(players)/10)): # So now we just need to navigate in the list containing each game
        for i in range(u*10,u*10+10):
          listGames[u].append(players[i]) # and here we add the players in a sorted way
      return listGames

    #The 12 first games with all the players leading to the 10 last players
    def tournamentPhase1(self, root): 
      players = self.getAllPlayersFromAVL(root,[])
      # 3 first games
      for i in range(3):
        self, root = randomizeScore(self, root)
      joueursRestant = self.createRankedgames(root)[1:]
      self, root = addPlayersFromRankedToAvlTree(joueursRestant)
      #Now we play and drop 10 players until 10 are left
      while len(joueursRestant) > 1:
        self, root = randomizeScore(self, root)
        joueursRestant = self.createRankedgames(root)[1:]
        self, root = addPlayersFromRankedToAvlTree(joueursRestant)
      return joueursRestant, self, root

    #The five last games with the top 10 players
    def tournamentPhase2(self, root, joueursRestant):
      for joueur in joueursRestant[0]:
        joueur["score"] = 0
        joueur["games"] = []
      #The five final games
      for i in range(5):
        self, root = randomizeScore(self, root)
      joueurFinale = sorted(joueursRestant[0], key = lambda player: player["score"])
      return joueurFinale, self, root



# add a list of players in a new AVL tree       
def add100PlayersToAvlTree(players):
    tree = AVLTree()
    root = None
    for player in players:         
        root = tree.insererNoeud(root, Player(player))
    return tree, root

# Takes a list of players which have been ranked (its a list of list) and add the players in a new AVL Tree
def addPlayersFromRankedToAvlTree(players):
    tree = AVLTree()
    root = None
    for liste in players:   
      for player in liste:      
        root = tree.insererNoeud(root, Player(player))
    return tree, root

#Function which creates a list of list of size "size"
def init_list_of_objects(size):
    list_of_objects = list()
    for i in range(0,size):
        list_of_objects.append(list())
    return list_of_objects

#Function which calculates the mean of the score of all games
def meanScore(games):
    score = int()
    for game in games:
        score += game
    score = score/len(games)
    return score

#Takes all the players from the three that exist into a list, add the randomized game score into the player games attribute, update their score, and then put them back in a tree, and then return the tree and the root
# The time complexity of this is O(n*log(n))
def randomizeScore(tree, root):
    players = []
    players = tree.getAllPlayersFromAVL(root, players) # We take all the players from the tree =>  O(n) operation
    root = None
    tree = AVLTree()
    for player in players:
        value = random.randrange(0, 13, 1)
        player["games"].append(value) 
        player["score"] = meanScore(player["games"])
        root = tree.insererNoeud(root, Player(player)) #We insert n times a value which means O(n*log(n))
    return tree, root


# used to display the top 10 players after they played the first 12 games
def phase1Display(joueursRestant):
  i= len(joueursRestant[0])
  print("Here is the ranking after the 12 first games leading to the top 10 players: \n")
  for joueur in joueursRestant[0]:
    print(str(i),"-",joueur["playername"],f'// score : {joueur["score"]}', joueur["games"])
    i-=1

# used to display the top 10 players after they played 5 last games
def podiumDisplay(joueurRestantFinale):
  print("\nLet's now reset their scores and play 5 games to get the final podium and the winner: \n")
  i= len(joueurRestantFinale)
  for joueur in joueurRestantFinale:
    print(str(i),"-",joueur["playername"],f'// score : {joueur["score"]}', joueur["games"])
    i-=1
  print("\nCongratulations to",joueurRestantFinale[9]["playername"], "who wins the game")
  


# MENU

In [4]:
menu = {}
menu['1']="Create 100 players with empty scores and games and display it" 
menu['2']="Create 100 players and randomize their score one time, display the empty tree, then display the randomized scored tree"
menu['3']="Create a list of random games from 100 a one-time-randomized-score players"
menu['4']="Create ranked games from a 100 players randomized scored tree, we display the randomized scored tree and then the playername and score of each players in games"
menu['5']="Create a tournament according to the rules, and display the podium"
menu['Enter anything else to exit']=""

while True: 
  options=menu.keys()
  sorted(options)
  for entry in options: 
    print (entry, menu[entry])
  selection=input("Please Select:") 

  if selection =='1': 
    players =[] # Let's create our list of 100 players
    for p in range(100):
      game = []
      player = {'playername' : f'Player_{p}', 'score' : 0, 'games' : game}
      players.append(player)
    tree, root = add100PlayersToAvlTree(players)  # Let's add our players in our AVL Tree
    print("\nVoici la liste des joueurs:\n",tree.getAllPlayersFromAVL(root,[]),"\n")    # Function which takes all the players from the AVL Tree and return a list, then, we  print the list
    root.display()  #Function of Miss Djebali which displays the tree
    print('\n')

  elif selection == '2': 
    players =[] # Let's create our list of 100 players
    for p in range(100):
      game = []
      player = {'playername' : f'Player_{p}', 'score' : 0, 'games' : game}
      players.append(player)
    tree, root = add100PlayersToAvlTree(players)  # Let's add our players in our AVL Tree
    root.display()
    print('\n')
    tree, root = randomizeScore(tree, root)  
    root.display()
    print('\n')

  elif selection == '3':
    players =[] # Let's create our list of 100 players
    for p in range(100):
      game = []
      player = {'playername' : f'Player_{p}', 'score' : 0, 'games' : game}
      players.append(player)
    tree, root = add100PlayersToAvlTree(players)  # Let's add our players in our AVL Tree
    tree, root = randomizeScore(tree, root)  
    root.display()
    games = tree.createRandomGames(root)
    i=1
    print('\n')
    for game in games:
      print("Game n°{} :".format(i), end=' ')
      i += 1
      for joueur in game:
        print(joueur["playername"], "(",joueur["score"],")", end=', ')
      print()
    print('\n')

  elif selection == '4': 
    players =[] # Let's create our list of 100 players
    for p in range(100):
      game = []
      player = {'playername' : f'Player_{p}', 'score' : 0, 'games' : game}
      players.append(player)
    tree, root = add100PlayersToAvlTree(players)  # Let's add our players in our AVL Tree
    print('\n')
    tree, root = randomizeScore(tree, root)  
    root.display()
    rankedgame = tree.createRankedgames(root)
    i=1
    print('\n')
    for game in rankedgame:
      print("Game n°{} :".format(i), end=' ')
      i += 1
      for joueur in game:
        print(joueur["playername"], "(",joueur["score"],")", end=', ')
      print()
    print('\n')

  elif selection == '5': 
    print('\n')
    players =[]
    for p in range(100):
        game = []
        player = {'playername' : f'Player_{p}', 'score' : 0, 'games' : game}
        players.append(player)
    tree, root = add100PlayersToAvlTree(players)  # add the players in an AVL Tree

    joueursRestant, tree, root = tree.tournamentPhase1(root)
    #print(joueursRestant)
    phase1Display(joueursRestant)
    #root.display()
    joueurRestantFinale, tree, root = tree.tournamentPhase2(root, joueursRestant)
    #print(joueurRestantFinale)
    #QUESTION 8
    podiumDisplay(joueurRestantFinale)
    #root.display()
    print('\n')
  else: 
    break

1 Create 100 players with empty scores and games and display it
2 Create 100 players and randomize their score one time, display the empty tree, then display the randomized scored tree
3 Create a list of random games from 100 a one-time-randomized-score players
4 Create ranked games from a 100 players randomized scored tree, we display the randomized scored tree and then the playername and score of each players in games
5 Create a tournament according to the rules, and display the podium
Enter anything else to exit 
Please Select:1

Voici la liste des joueurs:
 [{'playername': 'Player_63', 'score': 0, 'games': []}, {'playername': 'Player_31', 'score': 0, 'games': []}, {'playername': 'Player_15', 'score': 0, 'games': []}, {'playername': 'Player_7', 'score': 0, 'games': []}, {'playername': 'Player_3', 'score': 0, 'games': []}, {'playername': 'Player_1', 'score': 0, 'games': []}, {'playername': 'Player_0', 'score': 0, 'games': []}, {'playername': 'Player_2', 'score': 0, 'games': []}, {'pl

# QUESTION ANSWERING AND EXPLANATION OF EVERY STEPS OF THIS CODE - TESTS UNITAIRES

In [ ]:

# YOU DON'T HAVE TO USE THE LINES OF CODE DOWN BELOW, WE PUT IT HERE TO SHOW HOW THE CODE WOULD WORK, BUT EVERYTHING WORKS WITH THE MENU

#QUESTION 1 A data structure for the player and its score
'''
 it is a dictionnary composed of its name, its score (the mean of the score of all game), and the score of each game
 A dictionnary is a cleaner way to represent a player since the name of the parametres are defined (playername, score and games)
 and allows us to reach easily any of the parametres when we will construct the class Player to construct the Avl tree
 To be clear we could have used a list and just use indexes to find what we needed, but a dict is more comfortable
 thus, a dictionnary is far more scalable than a list, so that if we need to add somme parametres it will always be clear in the code
 Also with a dict, we will be able to sort our data on any parameter we want for example : sorting on score or sorting on player names
 Each player is stored in a list called players. This list will be the key to build our ALV Tree.
 We will explain later why we used the list called "players"
'''
#players =[] # Let's create our list of 100 players
#for p in range(100):
#    game = []
#    player = {'playername' : f'Player_{p}', 'score' : 0, 'games' : game}
#    players.append(player)
  
    
#QUESTION 2  Generate 100 Players in an AVL TREE
'''The most optimized data structure for the tournament is an AVL tree, as we saw it during class.
  Due to the balancing property, the insertion, deletion and search operations take O(log n)
  TO clarify the use of our functions, we first have saved the players in a list, and then stock them in our
  AVL Tree, this was just to be able to separate question 1 and 2 since this what is asked of us.
  But the most optimized thing to do in terms of space complexity is after creating player in question 1, adding it directly in the AVL Tree.
  Still the time complexity is the same and is O(n*log(n))
'''
#tree, root = add100PlayersToAvlTree(players)  # Let's add our players in our AVL Tree
#print(tree.getAllPlayersFromAVL(root,[]))    # Function which takes all the players from the AVL Tree and return a list, then, we  print the list
#root.display()  #Function of Miss Djebali which displays the tree


'''
The randomize function takes the tree and the root as arguments. It also returns the tree and the root.
It puts all the players from the AVL tree into a list (O(n) operation), add the randomized game score into the player games list, updates the score, and then insert them in a new AVL tree (here we do n times the inserting O(log(n)) operation so),
the time complexity for this loop is O(n*log(n))   and then return the tree and the root
Creating a new tree in order to fill it with the player's updated data is more optimized than stocking its values somewhere, deleting all its values and then inserting those back (stocking => O(n), deleting and insertting O(n*log(n)) ).
And it is also more optimized than browsing every node of the AVL Tree in order to simply update the scores in terms of time complexity because by doing it, we would have to sort the whole tree again, which means balancing
the tree, which is very difficult in that case and time consumming.

We could also have used updateScoresRandomlyAndInsertInNewTree() which browse the AVL Tree (O(n)) and updates the player while browing and add it to a new AVL Tree (O(log(n)) without using a list between. 
The time complexity would be also O(n*log(n)) but we would'nt use a list so the space complexity would be better

We can use both methods

So finnaly our time complexity for randomizing players score and updating the AVL Tree (database) is n*log(n)

'''
#QUESTION 3/4  Randomize and update 
#put the old score in a list, then add a random number to the score of each player, then insert it back to the tree
#tree, root = randomizeScore(tree, root)  
# OR
#root, tree = tree.updateScoresRandomlyAndInsertInNewTree(root, AVLTree(), None)
#root.display()

'''
We create a list of list of the number of games that will be played (ex : for 90 players => 9 games, so a list of 9 lists).
We create a list of indices going from 0 to 99, which will be used to select the players only once.
We select a random “number” - which is in the range of the length of the list “indices” -  and take the “value” of the player in the list "indices" at the index “number”, and then we will delete the “value” from the list “indices” so that we won't choose this player again (in fact we pop the value so that it gives us the value and delete it at the same time).
We put random and different 10 players in a list and add this list to the list containing all the games and we do it until there are no players left.
This way we are sure there is no 2 players who are the same in all the games.
'''
#QUESTION 5  Create random games based on the DB
#Il faire une liste comprenant une liste de 10 joueurs, qui soit scalable en fonction du nombre de joueurs restant
#players = tree.createSortedPlayers(root,[])
#print(players)

'''
We first create a list of list of the number of games that will be played (ex : for 90 players => 9 games, so a list of 9 lists)
Thanks to an in order traversal, we create a list of players in their ascending order
So we just need to navigate in the sorted list containing each player, and add them into games of 10
'''
#QUESTION 6  Create games based on ranking
#rankedgame = tree.createRankedgames(root)
#print(rankedgame)


'''
We call the function tournamentPhase1 which simulate the 12 first games. 
There is our 100 players. We play the 3 first games with no eliminations. At the end of it we drop the 10 worst players.
Then we will play a game and drop the 10 worst players until there are only 10 players left.
We then print those players to show their scores and games.
Then we call the function tournamentPhase2 which simulates the 5 last games.
We reset every players scores and games, and then simulate 5 games.

'''
#QUESTION 7  Drop the players and play until the last 10 players
#We begin with a fresh start
'''players =[]
for p in range(100):
    game = []
    player = {'playername' : f'Player_{p}', 'score' : 0, 'games' : game}
    players.append(player)
'''
#tree, root = add100PlayersToAvlTree(players)  # add the players in an AVL Tree
#joueursRestant, tree, root = tree.tournamentPhase1(root)
#print(joueursRestant)
#phase1Display(joueursRestant)
#root.display()

#joueurRestantFinale, tree, root = tree.tournamentPhase2(root, joueursRestant)
#print(joueurRestantFinale)

'''
We finally print the podium.
'''
#QUESTION 8
#podiumDisplay(joueurRestantFinale)
#root.display()

